In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm
import numpy as np
import os
from tqdm import trange
import pandas as pd

from utils import plot_me, smooth_traces, generate_track_interactions
from track import Track

Autosaving every 180 seconds


/home/cat/miniconda3/envs/sleap_new/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
##########################################################################
####################### HELPER FUNCTIONS #################################
##########################################################################

#
def parse_spreadsheet_return_unique_vids_and_starts_ends(header, 
                                                         dataframe):

    # get animal ids first
    animal_labels = []
    for k in range(1,len(header),2):
        temp = header[k]
        animal_labels.append(temp)

    #print ("animal_labels: ", animal_labels)

    # get starts and ends next
    starts_ends = []
    video_names = []
    animal_ids = []
    data_npy = dataframe.to_numpy()
    #print (data_npy.shape)

    #
    for k in range(data_npy.shape[0]):
        vid_name = data_npy[k][0]
        for p in range(1,data_npy.shape[1],2):
            #print (data_npy[k,p])
            if data_npy[k,p]!=0:
                starts_ends.append([data_npy[k,p],data_npy[k,p+1]])
                video_names.append(vid_name)
                animal_ids.append(animal_labels[p//2])

    #
    video_names = np.array(video_names)
    starts_ends = np.array(starts_ends)
    animal_ids = np.array(animal_ids)

    # #
    # for k in range(len(starts_ends)):
    #     print (video_names[k], 
    #            starts_ends[k],
    #            animal_ids[k])

    # gather all video-animal pairs that are the same 
    unique_vids = np.unique(video_names)
    unique_ids = np.unique(animal_ids)
    #print (unique_ids)

    # 
    ctr=0
    final_arrays= []
    final_arrays.append([])
    for unique_vid in unique_vids:
        for unique_id in unique_ids:
           
            
            # loop over all vids
            se_array = []
            for k in range(starts_ends.shape[0]):
                if video_names[k] == unique_vid:
                    if animal_ids[k] == unique_id:
                        se_array.append(starts_ends[k])
            #
            if len(se_array)>0:
                final_arrays[ctr].append(unique_vid)
                final_arrays[ctr].append(unique_id)
                final_arrays[ctr].append(np.vstack(se_array).squeeze())
                final_arrays.append([])
                ctr+=1
    
    if len(final_arrays[-1])==0:
        del(final_arrays[-1])

    # 
    print ("                                UNIQUE VIDEO ID           ANIMAL IDS      STARTS-ENDS ARRAYS")
    for k in range(len(final_arrays)):
        print (" unique video - pair: #", k, ": ", final_arrays[k])

    return final_arrays

#
def make_pairwise_npy_files_for_simba(final_arrays,
                                     root_dir):
    #
    for dataset in final_arrays:

        #
        fname = dataset[0]

        #
        animal_ids = np.int32([dataset[1][0],dataset[1][2]])

        #
        starts_ends = dataset[2]

        fname_out = os.path.join(root_dir+'/pairwise'+
                            fname+"_animals_"+str(animal_ids[0])+"_"+str(animal_ids[1])+'.npy')
        
        #
        if os.path.exists(fname_out):
            continue
        
        #
        print ("PROCESSING: ")
        print (fname)
        print (animal_ids)
        print (starts_ends)
        
        #
        track = Track(os.path.join(root_dir, 
                                   fname+'_compressed_Day.slp'))
        track.animal_ids = animal_ids
        track.tracks_names = ['female','male','pup1','pup2','pup3','pup4']
        track.recompute_spine_centres=True
        track.verbose = True                         # gives additional printouts
        track.n_animals = len(track.animal_ids)      # number of animals
        track.filter_width = 10                      # this is the median filter width in frames; e.g. 10 ~=0.4 seconds
                                                     # higher values provide more stability, but less temporally precise locations
        # 
        track.load_tracks()

        #
        data_out = np.zeros((track.tracks_spine.shape[0],
                             2,2,2))

        #
        data_out[:,:,:,0] = track.tracks_spine[:,animal_ids]

        #
        try:
            for se in starts_ends:
                data_out[int(se[0]):int(se[1]),:,:,1]=1
        except:
            print ("single exmaple only")
            data_out[int(starts_ends[0]):int(starts_ends[1]),:,:,1]=1

        #
        np.save(fname_out, data_out)
        
        ####################################################
        ############# MAKE ANIMAL ANGLES ###################
        ####################################################
        
        track.smooth_angles = True
        track.compute_angles_from_features(fname_out)
        np.save(fname_out[:-4]+"_angles.npy",
                track.angles)    
        
        #
        print ("")


In [ ]:

#
def make_pairwise_npy_files_for_simba_single_file(fname_in,
                                                   root_dir):
   

    dataset = 

        #
        fname = dataset[0]

        #
        animal_ids = np.int32([dataset[1][0],dataset[1][2]])

        #
        starts_ends = dataset[2]

        fname_out = os.path.join(root_dir+'/pairwise'+
                            fname+"_animals_"+str(animal_ids[0])+"_"+str(animal_ids[1])+'.npy')
        
        #
        if os.path.exists(fname_out):
            continue
        
        #
        print ("PROCESSING: ")
        print (fname)
        print (animal_ids)
        print (starts_ends)
        
        #
        track = Track(os.path.join(root_dir, 
                                   fname+'_compressed_Day.slp'))
        track.animal_ids = animal_ids
        track.tracks_names = ['female','male','pup1','pup2','pup3','pup4']
        track.recompute_spine_centres=True
        track.verbose = True                         # gives additional printouts
        track.n_animals = len(track.animal_ids)      # number of animals
        track.filter_width = 10                      # this is the median filter width in frames; e.g. 10 ~=0.4 seconds
                                                     # higher values provide more stability, but less temporally precise locations
        # 
        track.load_tracks()

        #
        data_out = np.zeros((track.tracks_spine.shape[0],
                             2,2,2))

        #
        data_out[:,:,:,0] = track.tracks_spine[:,animal_ids]

        #
        try:
            for se in starts_ends:
                data_out[int(se[0]):int(se[1]),:,:,1]=1
        except:
            print ("single exmaple only")
            data_out[int(starts_ends[0]):int(starts_ends[1]),:,:,1]=1

        #
        np.save(fname_out, data_out)
        
        ####################################################
        ############# MAKE ANIMAL ANGLES ###################
        ####################################################
        
        track.smooth_angles = True
        track.compute_angles_from_features(fname_out)
        np.save(fname_out[:-4]+"_angles.npy",
                track.angles)    
        
        #
        print ("")
        
        
#
fname = '/home/cat/Downloads/approach/2020_07_28_17_11_05_340439_compressed_Day_0_1_cleaned.npy'

#
make_pairwise_npy_files_for_simba_single_file

In [52]:
##############################################################
################# LOAD DATA FRAME AND ID PAIRS ###############
##############################################################

# these are the ids of the gerbils that Lisa and Jennifer decided to label
# ids 0->5 correspond to female->pup4
id_pairs = [
    [0,1],
    [1,0],
    [4,5],
    [5,4],
    [0,2],
    [2,0],
    [1,3],
    [3,1]
]

fname = "/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/simba/approach.csv"

#
dataframe = pd.read_csv(fname)
header = dataframe.columns.tolist()

#
# dataframe



In [ ]:
#########################################################################
################# CONVERT SPREADSHEET TO 2 COLUMN VERSION ###############
#########################################################################
final_arrays = parse_spreadsheet_return_unique_vids_and_starts_ends(header, dataframe)
            

In [56]:
###########################################################################################
########## GENERATE .NPY FILES FROM .SLP FOR EACH UNIQUE VID-PAIR AND NUMPY ARRAYS ########
###########################################################################################

##############################################################
root_dir = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/simba'

# break
make_pairwise_npy_files_for_simba(final_arrays,
                                     root_dir)  
print ("DONE...")

PROCESSING: 
2020_08_03_19_13_00_372474
[0 2]
[[23842 23853]
 [ 5046  5073]]
... npy missing, converting...
... h5 file missing, converting now...
... done loading slp

Exporting to SLEAP Analysis file...
	track_names: 6
	node_names: 6
	edge_names: 5
	edge_inds: 5
	tracks: (28802, 6, 2, 6)
	track_occupancy: (6, 28802)
	point_scores: (28802, 6, 6)
	instance_scores: (28802, 6)
	tracking_scores: (28802, 6)
	labels_path: None
	video_path: /vast/cm5635/cohort2/videos/2020_08_03_19_13_00_372474_compressed_defished_shrink_cropped.mp4
	video_ind: 0
	provenance: {}
Saved as /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/simba/2020_08_03_19_13_00_372474_compressed_Day.h5
... done loading h5
group2:  <HDF5 dataset "tracks": shape (6, 2, 6, 28802), type "<f8">
... done loading npy
(28802, 2, 2, 2)


100%|██████████████████████████████████| 28802/28802 [00:01<00:00, 14751.22it/s]



PROCESSING: 
2020_08_03_19_13_00_372474
[1 3]
[[ 7931  7944]
 [11571 11587]
 [16077 16090]]
(28802, 2, 2, 2)


100%|██████████████████████████████████| 28802/28802 [00:02<00:00, 13450.00it/s]



PROCESSING: 
2020_08_03_19_13_00_372474
[2 0]
[2407 2418]
single exmaple only
(28802, 2, 2, 2)


100%|██████████████████████████████████| 28802/28802 [00:02<00:00, 13813.28it/s]



PROCESSING: 
2020_08_03_19_13_00_372474
[5 4]
[[  59   71]
 [5029 5039]]
(28802, 2, 2, 2)


100%|██████████████████████████████████| 28802/28802 [00:01<00:00, 15154.54it/s]


DONE...


In [6]:
data = np.load('/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/simba/2020_08_01_11_02_57_239373/2020_08_01_11_02_57_239373_animals_5_4.npy')

print ("# frames, # animals, x-y, locations and behavior yes/no")
print (data.shape)

frame_id = 20280  #<--- example of on behavior
animal_id = 0
print ("frame: ", frame_id, "animal id: ", animal_id, ": locations: ", data[frame_id,animal_id,:,0])
print ("frame: ", frame_id, "animal id: ", animal_id, ": behavior state: ", data[frame_id,animal_id,:,1])

# frames, # animals, x-y, locations and behavior yes/no
(28802, 2, 2, 2)
frame:  20280 animal id:  0 : locations:  [544.71557617 563.74786377]
frame:  20280 animal id:  0 : behavior state:  [1. 1.]
